In [1]:
import os
import re
import shutil
from PIL import Image, ImageDraw
import numpy as np

In [2]:
FRAMES = 'C:\\_Repositories\\Kamaz_Control\\data\\train_'
MARKING = 'C:\\_Repositories\\Kamaz_Control\\data\\_marked_files'

In [3]:
jpg_folders = []
for root, _, files in os.walk(FRAMES):
    for file in files:
        if file.endswith('.jpg'):
            s = os.path.dirname(os.path.join(root, file))
            if not s in jpg_folders:
                jpg_folders.append(s)

In [4]:
SELECTION_FRAMES = 'C:\\_Repositories\\Kamaz_Control\\data\\selection\\frames'
SELECTION_MARKING = 'C:\\_Repositories\\Kamaz_Control\\data\\selection\\marks'
SELECTION_VISIALIZATION = 'C:\\_Repositories\\Kamaz_Control\\data\\selection\\visualization'

In [5]:
def create_embedded_folder(path_to_dir):
    if not os.path.exists(path_to_dir):
        os.makedirs(path_to_dir)

In [6]:
create_embedded_folder(SELECTION_FRAMES)
create_embedded_folder(SELECTION_MARKING)
create_embedded_folder(SELECTION_VISIALIZATION)

In [7]:
def get_all_files(data_path, format_file):
    res = []
    for root, _, files in os.walk(data_path):
        for file in files:
            if file.endswith(format_file):
                 res.append(file)
    return res

In [8]:
def get_class_number(marked_file):
    if marked_file.find('ekranturb') >= 0:
        return 'ekranturb', '0'
        #return '0'
    if marked_file.find('kkomp') >= 0:
        return 'kkomp', '1'
        #return '1'
    if marked_file.find('kol') >= 0:
        return 'kol', '2'
        #return '2'
    if marked_file.find('krysh') >= 0:
        return 'krysh', '3'
        #return '3'
    if marked_file.find('perehodnik') >= 0:
        return 'perehodnik', '4'
        #return '4'
    if marked_file.find('pod') >= 0:
        return 'pod', '5'
        #return '5'
    if marked_file.find('turbokompressor') >= 0:
        return 'turbokompressor', '7'
        #return '7'
    if marked_file.find('turb') >= 0:
        return 'turb', '6'
        #return '6'
    if marked_file.find('usel') >= 0:
        return 'usel', '8'
        #return '8'
    if marked_file.find('val') >= 0:
        return 'val', '9'
        #return '9'
    return ''

In [9]:
def xml_content(folder_name, file_name, class_number, x1, y1, x2, y2):
    return '<?xml version="1.0" encoding="utf-8"?>\n'  + \
           '<annotation>\n' + \
           '  <folder>VOC0712</folder>\n' + \
           '  <filename>{}</filename>\n'.format(file_name) + \
           '  <source>\n' + \
           '    <database>KAMAZ Database</database>\n' + \
           '    <annotation>KAMAZ</annotation>\n' + \
           '    <image>flickr</image>\n' + \
           '    <flickrid>0</flickrid>\n' + \
           '  </source>\n' + \
           '  <owner>\n' + \
           '    <flickrid>Unspecified</flickrid>\n' + \
           '    <name>Unspecified</name>\n' + \
           '  </owner>\n' + \
           '  <size>\n' + \
           '    <width>1920</width>\n' + \
           '    <height>1080</height>\n' + \
           '    <depth>3</depth>\n' + \
           '  </size>\n' + \
           '  <segmented>0</segmented>\n' + \
           '  <object>\n' + \
           '    <name>{}</name>\n'.format(class_number) + \
           '    <pose>Unspecified</pose>\n' + \
           '    <truncated>0</truncated>\n' + \
           '    <difficult>0</difficult>\n' + \
           '    <bndbox>\n' + \
           '      <xmin>{}</xmin>\n'.format(x1) + \
           '      <ymin>{}</ymin>\n'.format(y1) + \
           '      <xmax>{}</xmax>\n'.format(x2) + \
           '      <ymax>{}</ymax>\n'.format(y2) + \
           '    </bndbox>\n' + \
           '  </object>\n' + \
           '</annotation>'

In [10]:
def create_marking_xml(filename, foldername, framename, class_number, coords):
    with open(filename, 'w') as output_file:
        str_data = xml_content(foldername, framename, class_number, coords[0], coords[1], coords[2], coords[3])
        output_file.write(str_data)

In [11]:
def create_marking_picture(file_pic, coords, marked_file):
    img = Image.open(file_pic)
    draw = ImageDraw.Draw(img)
    draw.rectangle(coords, outline='red')
    img.save(marked_file)

In [12]:
for jpg_folder in jpg_folders:
    pics = get_all_files(jpg_folder, '.jpg')
    txt_filename = 'marked_' + re.sub('\.', '', str.lower(os.path.basename(jpg_folder))) + '.txt'
    txt_file = os.path.join(MARKING, txt_filename)
    marks = np.loadtxt(txt_file, dtype=int, delimiter=' ')
    for i in range(len(pics)):
        jpg_file = pics[i]
        coords = list(marks[i, :])
        class_name, class_number = get_class_number(txt_file)
        video_number = '{:0>2}'.format(int(re.sub('[^0-9]', '', txt_filename)))
        file_name = '{}_{}_{}'.format(class_number, video_number, jpg_file.split('.')[0])
        src_file = os.path.join(jpg_folder, jpg_file)
        dst_file = os.path.join(SELECTION_FRAMES, file_name + '.jpg')
        #shutil.copyfile(src_file, dst_file)
        xml_file = os.path.join(SELECTION_MARKING, file_name + '.xml')
        create_marking_xml(xml_file, os.path.basename(SELECTION_FRAMES), file_name + '.jpg', class_name, coords)
        marked_file = os.path.join(SELECTION_VISIALIZATION, file_name + '.jpg')
        #create_marking_picture(src_file, coords, marked_file)